# TVA sur les biens alimentaires

In [1]:
from __future__ import division

import os
import pandas as pd
import pkg_resources
import seaborn

from openfisca_france_indirect_taxation.surveys import SurveyScenario
from openfisca_france_indirect_taxation.build_survey_data.calibration_aliss import get_adjusted_input_data_frame


In [ ]:
year = 2014
data_year = 2011
reform_key = 'aliss_tva_sociale'
scenario_kwargs = dict(year = year, data_year = 2011, reform_key = reform_key)
survey_scenario = SurveyScenario.create(**scenario_kwargs)
# 

In [8]:
adjusted_scenario_kwargs = dict(scenario_kwargs)
adjusted_scenario_kwargs.update(dict(
    data_year = None,
    input_data_frame = get_adjusted_input_data_frame(reform_key = reform_key[6:])
    ))
adjusted_survey_scenario = SurveyScenario.create(**adjusted_scenario_kwargs) 

kantar_expenditures_total:  52.035641344
bdf_expenditures_total:  52.218515456
population_kantar_total:  11968.2298584
bdf_expenditures_total:  126.629058796
0 0 poste_01_1_1_4_3
0 0 poste_01_1_1_1_1
0 0 poste_01_1_1_4_2
0 0 poste_01_1_1_3_3
0 0 poste_01_1_1_4_1
0 0 poste_01_1_2_1_1
0 0 poste_01_1_2_4_1
0 0 poste_01_1_2_3_1
0 0 poste_01_1_2_5_1
0 0 poste_01_1_2_4_2
0 0 poste_01_1_2_6_3
0 0 poste_01_1_2_6_1
0 0 poste_01_1_3_1_1
0 0 poste_01_1_3_2_1_a
0 0 poste_01_1_3_1_2
0 0 poste_01_1_3_2_1_b
0 0 poste_01_1_3_2_2
0 0 poste_01_1_4_1_1_a
0 0 poste_01_1_4_1_1_b
0 0 poste_01_1_4_1_1_c
0 0 poste_01_1_4_2_1
0 0 poste_01_1_4_3_1
0 0 poste_01_1_4_2_2
0 0 poste_01_1_4_4_1
0 0 poste_01_1_5_1_1
0 0 poste_01_1_5_2_2_a
0 0 poste_01_1_5_2_1_a
0 0 poste_01_1_5_2_1_b
0 0 poste_01_1_5_2_2_b
0 0 poste_01_1_6_1_1_a
0 0 poste_01_1_6_1_1_b
0 0 poste_01_1_6_1_1_c
0 0 poste_01_1_6_1_1_d
0 0 poste_01_1_6_2_1_a
0 0 poste_01_1_6_2_1_b
0 0 poste_01_1_6_2_1_c
0 0 poste_01_1_6_2_1_d
0 0 poste_01_1_6_2_1_e
0 0 post

In [9]:
alimentation_domicile_hors_alcool = [
    "depenses_ht_{}".format(key) for key in survey_scenario.tax_benefit_system.column_by_name.keys() \
    if key.startswith(u'poste_01')
    ]
alimentation_domicile = alimentation_domicile_hors_alcool + [
    'depenses_biere',
    'depenses_vin',
    'depenses_alcools_forts'
    ]

In [10]:
variables = alimentation_domicile + [
    'poste_agrege_01',
    'poste_agrege_02', 
    'depenses_ht_tva_taux_super_reduit',
    'depenses_ht_tva_taux_reduit',
    'depenses_ht_tva_taux_intermediaire',
    'depenses_ht_tva_taux_plein',
    'tva_taux_super_reduit',
    'tva_taux_reduit',
    'tva_taux_intermediaire',
    'tva_taux_plein',
    'tva_total',
    ]

In [11]:
pivot_table = pd.DataFrame()
reference_pivot_table = pd.DataFrame()
adjusted_pivot_table = pd.DataFrame()
aggregates = dict()
reference_aggregates = dict()
adjusted_aggregates = dict()

for variable in variables:
    pivot_table = pd.concat([
        pivot_table,
        survey_scenario.compute_pivot_table(values = [variable], columns = ['niveau_vie_decile'])
        ])
    reference_pivot_table = pd.concat([
        reference_pivot_table,
        survey_scenario.compute_pivot_table(values = [variable], columns = ['niveau_vie_decile'],
                                           reference = True)
        ])
    adjusted_pivot_table = pd.concat([
        adjusted_pivot_table,
        adjusted_survey_scenario.compute_pivot_table(values = [variable], columns = ['niveau_vie_decile'])
        ])
    aggregates[variable] = survey_scenario.compute_aggregate(variable) / 1e9
    reference_aggregates[variable] = survey_scenario.compute_aggregate(variable, reference = True) / 1e9
    adjusted_aggregates[variable] = adjusted_survey_scenario.compute_aggregate(variable) / 1e9

In [12]:
assert sum(aggregates[aliment] - reference_aggregates[aliment] for aliment in alimentation_domicile) == 0

In [13]:
assert aggregates['poste_agrege_02'] - reference_aggregates['poste_agrege_02'] < 1e-7

In [14]:
for key in sorted(aggregates.keys()):
    if round(aggregates[key] - reference_aggregates[key], 3) != 0:
         print key, ': ', round(aggregates[key], 3), ' - ', round(reference_aggregates[key], 3), ' = ', round(aggregates[key] - reference_aggregates[key], 3)

depenses_ht_tva_taux_intermediaire :  183.132  -  75.594  =  107.538
depenses_ht_tva_taux_plein :  311.769  -  315.773  =  -4.004
depenses_ht_tva_taux_reduit :  24.994  -  128.528  =  -103.534
poste_agrege_01 :  128.399  -  124.14  =  4.259
tva_taux_intermediaire :  18.313  -  7.559  =  10.754
tva_taux_plein :  62.354  -  63.155  =  -0.801
tva_taux_reduit :  1.375  -  7.069  =  -5.694
tva_total :  82.183  -  77.924  =  4.259


In [15]:
for key in sorted(aggregates.keys()):
    if round(adjusted_aggregates[key] - aggregates[key], 3) != 0:
         print key, ': ', round(adjusted_aggregates[key], 3), ' - ', round(aggregates[key], 3), ' = ', round(adjusted_aggregates[key] - aggregates[key], 3)

depenses_alcools_forts :  0.115  -  2.932  =  -2.817
depenses_biere :  0.015  -  1.301  =  -1.286
depenses_ht_poste_01_10_1 :  2.868  -  2.924  =  -0.056
depenses_ht_poste_01_10_2 :  1.401  -  1.427  =  -0.025
depenses_ht_poste_01_1_1_1_1 :  2.921  -  13.376  =  -10.455
depenses_ht_poste_01_1_1_3_3 :  0.153  -  3.73  =  -3.577
depenses_ht_poste_01_1_1_4_1 :  0.018  -  1.488  =  -1.47
depenses_ht_poste_01_1_1_4_2 :  0.022  -  1.615  =  -1.593
depenses_ht_poste_01_1_1_4_3 :  0.011  -  0.851  =  -0.841
depenses_ht_poste_01_1_2_1_1 :  0.473  -  7.457  =  -6.984
depenses_ht_poste_01_1_2_3_1 :  0.02  -  1.075  =  -1.055
depenses_ht_poste_01_1_2_4_1 :  0.038  -  2.176  =  -2.138
depenses_ht_poste_01_1_2_4_2 :  0.488  -  8.374  =  -7.886
depenses_ht_poste_01_1_2_5_1 :  0.199  -  4.766  =  -4.566
depenses_ht_poste_01_1_2_6_1 :  0.002  -  0.471  =  -0.469
depenses_ht_poste_01_1_2_6_3 :  0.08  -  2.943  =  -2.862
depenses_ht_poste_01_1_3_1_1 :  0.091  -  2.841  =  -2.75
depenses_ht_poste_01_1_3_1

In [ ]:
from openfisca_france_indirect_taxation.model.consommation.postes_coicop import get_legislation_data_frames
categories_fiscales_data_frame, codes_coicop_data_frame = get_legislation_data_frames()

In [ ]:
aggregates['poste_agrege_01'] - reference_aggregates['poste_agrege_01']

In [ ]:
pivot_table -pivot_table_reference